In [1]:
import os
import pdb
import _pickle as cPickle
import time

import numpy as np
import tensorflow as tf

from collections import defaultdict, Counter
from scipy.special import gammaln

# data 

In [2]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

flags = tf.app.flags

# flags.DEFINE_string('data_path', 'data/synthetic/instances_ncrp_9.pkl', 'path of data')
flags.DEFINE_string('data_path', 'data/bags/instances.pkl', 'path of data')

flags.DEFINE_integer('n_depth', 3, 'path of data')

flags.DEFINE_string('f', '', 'kernel')

config = flags.FLAGS

In [3]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.data_path,'rb'))
docs_bow = [instance.bow for instance in instances_train]
docs_raw = [[[bow_index]*int(doc_bow[bow_index]) for bow_index in np.where(doc_bow > 0)[0]] for doc_bow in docs_bow]
docs_words = [[idx for idxs in doc for idx in idxs] for doc in docs_raw][:5000]
words = [word for doc_words in docs_words for word in doc_words]

In [4]:
test_docs_bow = [instance.bow for instance in instances_test]
test_docs_raw = [[[bow_index]*int(test_doc_bow[bow_index]) for bow_index in np.where(test_doc_bow > 0)[0]] for test_doc_bow in test_docs_bow]
test_docs_words = [[idx for idxs in doc for idx in idxs] for doc in test_docs_raw][:5000]

In [5]:
n_doc = len(docs_words)
n_vocab = len(np.unique(words))
n_words = len(words)
assert n_vocab == len(bow_idxs)
n_doc, n_vocab, n_words

(5000, 1035, 92009)

# method

## assign docs to tree

In [132]:
class Topic:
    def __init__(self, idx, sibling_idx, parent, depth, n_doc, n_vocab):
        self.idx = idx
        self.sibling_idx = sibling_idx
        self.parent = parent
        self.children = []
        self.depth = depth
        self.cnt_doc = 0
        self.n_doc = n_doc
        self.n_vocab = n_vocab
        self.cnt_words = np.zeros(n_vocab) # Number of Words over Documents
        self.verbose = verbose
    
    def sample_child(self, doc):
        s_child_prior = self.get_s_child_prior(gam)
        s_child_likelihood = self.get_s_child_likelihood(doc, eta)
        p_child = np.array(s_child_prior * s_child_likelihood) / np.sum(s_child_prior * s_child_likelihood)
        
        child_index = np.random.multinomial(1, p_child).argmax()
        if verbose: print('Depth: ', self.depth, 'p_child: ', p_child, 'selected:', child_index)
        
        child = self.children[child_index] if child_index < len(self.children) else self.add_child()
        return child
    
    def init_sample_child(self):
        s_child_prior = self.get_s_child_prior(gam)
        p_child = np.array(s_child_prior) / np.sum(s_child_prior)
        
        child_index = np.random.multinomial(1, p_child).argmax()
        if verbose: print('Depth: ', self.depth, 'p_child: ', p_child, 'selected:', child_index)
        
        child = self.children[child_index] if child_index < len(self.children) else self.add_child()
        return child
    
    def get_probs_child(self, doc):
        s_child_prior = self.get_s_child_prior(gam)
        s_child_likelihood = self.get_s_child_likelihood(doc, eta)
        p_child = np.array(s_child_prior * s_child_likelihood) / np.sum(s_child_prior * s_child_likelihood)
        return p_child
    
    def get_s_child_prior(self, gam):
        s_child_prior = [child.cnt_doc for child in self.children]
        s_child_prior += [gam]
        return s_child_prior
    
    def get_s_child_likelihood(self, doc, eta):
        if len(self.children) > 0:
            children_cnt_words = np.concatenate([np.array([child.cnt_words for child in self.children]), np.zeros([1, self.n_vocab])], 0) # (Children+1) x Vocabulary
        else:
            children_cnt_words = np.zeros([1, self.n_vocab]) # (Children+1) x Vocabulary
        
        cnt_words_doc = doc.cnt_words[None, :] # 1 x Vocabulary

        logits_likelihood = gammaln(np.sum(children_cnt_words, -1) + n_vocab*eta) \
                            - np.sum(gammaln(children_cnt_words + eta), -1) \
                            - gammaln(np.sum(children_cnt_words + cnt_words_doc, -1) + n_vocab*eta) \
                            + np.sum(gammaln(children_cnt_words + cnt_words_doc + eta), -1)
        s_child_likelihood = np.exp(logits_likelihood)
        return s_child_likelihood
    
    def add_child(self):
        sibling_idx = max([child.sibling_idx for child in self.children]) + 1 if len(self.children) > 0 else 1
        idx = self.idx + '-' + str(sibling_idx)
        depth = self.depth+1
        child = Topic(idx=idx, sibling_idx=sibling_idx, parent=self, depth=depth, n_doc=self.n_doc, n_vocab=self.n_vocab)
        self.children += [child]
        return child
    
    def get_children(self):
        sibling_idx = max([child.sibling_idx for child in self.children]) + 1 if len(self.children) > 0 else 1
        idx = self.idx + '-' + str(sibling_idx)
        depth = self.depth+1
        child = Topic(idx=idx, sibling_idx=sibling_idx, parent=self, depth=depth, n_doc=self.n_doc, n_vocab=self.n_vocab)
        children = self.children + [child]
        return children
    
    def delete_topic(self):
        self.parent.children.remove(self)

In [131]:
class Doc:
    def __init__(self, idx, words, bow, n_depth):
        self.idx = idx
        self.words = words
        self.cnt_words = bow
        assert len(words) == np.sum(bow)
        
        self.topics = [] # Depth
        self.word_depths = [] # Word Indices
        self.depth_cnt_words = np.zeros([n_depth, n_vocab])
        
        # for test
        self.probs_paths = []
        self.probs_depths = []
        
    def get_probs_depth(self, word_idx):
        s_docs = np.sum(self.depth_cnt_words, -1) + alpha # Depth
        s_words = np.array([topic.cnt_words[word_idx] for topic in self.topics]) + eta # Depth
        z_words = np.array([np.sum(topic.cnt_words) for topic in self.topics]) + n_vocab*eta # Depth
        assert s_docs.shape == s_words.shape == z_words.shape

        s_depths = s_docs*s_words/z_words
        p_depths = s_depths/np.sum(s_depths) # Depth
        return p_depths
    
    def sample_depth(self, word_idx):
        prob_depths = self.get_probs_depth(word_idx)
        word_depth = np.argmax(np.random.multinomial(1, prob_depths))
        return word_depth

## sample doc path

$$p({\bf c}_{m}\hspace{0.5ex}|\hspace{0.5ex}{\bf w}, {\bf c}_{-m}, {\bf z})\propto p({\bf w}_{m}\hspace{0.5ex}|\hspace{0.5ex}{\bf c}, {\bf w}_{-m}, {\bf z})\cdot p({\bf c}_{m}\hspace{0.5ex}|\hspace{0.5ex}{\bf c}_{-m})$$

$$p({\bf w}_{m}\hspace{0.5ex}|\hspace{0.5ex}{\bf c}, {\bf w}_{-m}, {\bf z})=\prod_{\ell=1}^{L}\left(\frac{\Gamma(n_{c_{m,\ell},-m}^{(\cdot)}+W\eta)}{\prod_{w}\Gamma(n_{c_{m,\ell},-m}^{(w)}+\eta)}\frac{\prod_{w}\Gamma(n_{c_{m,\ell},-m}^{(w)}+n_{c_{m,\ell},m}^{(w)}+\eta)}{\Gamma(n_{c_{m,\ell},-m}^{(\cdot)}+n_{c_{m,\ell},m}^{(\cdot)}+W\eta)}\right)$$

In [133]:
def init_doc_topics(docs, topic_root):
    for doc in docs:
        topic = topic_root
        doc.topics = [topic]
        topic.cnt_doc += 1 # increment count of docs

        for depth in range(1, n_depth):
            topic = topic.init_sample_child()
            doc.topics += [topic]
            topic.cnt_doc += 1 # increment count of docs

In [134]:
def sample_doc_topics(docs, topic_root):
    for doc in docs:
        for depth in range(1, n_depth):
            topic = doc.topics[depth]
            topic.cnt_doc -= 1 # decrement count of docs
            assert topic.cnt_doc >= 0
            topic.cnt_words -= doc.depth_cnt_words[depth] # decrement count of words
            assert np.min(topic.cnt_words) >= 0
            
            if topic.cnt_doc == 0: 
                topic.delete_topic()
                assert np.sum(topic.cnt_words) == 0

        topic = topic_root
        doc.topics = [topic]
        for depth in range(1, n_depth):
            topic = topic.sample_child(doc)
            doc.topics += [topic]
            topic.cnt_doc += 1 # increment count of docs
            topic.cnt_words += doc.depth_cnt_words[depth] # increment count of words

In [135]:
def get_prob_doc_topics(docs, topic_root):
    for doc in docs:
        topic = topic_root
        doc.probs_paths.append({topic: 1.})
        for depth in range(1, n_depth):
            probs_path = {}
            for topic, prob_path in doc.probs_paths[-1].items():
                topics_child = topic.get_children()
                probs_child = topic.get_probs_child(doc)
                probs_path_child = prob_path * probs_child
                assert len(topics_child) == len(probs_path_child)
                for topic_child, prob_path_child in zip(topics_child, probs_path_child):
                    probs_path[topic_child] = prob_path_child
            doc.probs_paths.append(probs_path)

## assign words to topics

\begin{align*}
p(z_{i}=j\hspace{0.5ex}|\hspace{0.5ex}{\bf z}_{-i},{\bf w})\propto (n_{-i,j}^{(d_{i})}+\alpha)\frac{n_{-i,j}^{(w_{i})}+\eta}{n_{-i,j}^{(\cdot)}+W\eta}
\end{align*}

In [136]:
def init_word_topics(docs, train=True):
    for doc in docs:
        if doc.idx % 1000 == 0: print(doc.idx, end=' ')
        for word_index, word_idx in enumerate(doc.words):
            # sample depth of word
            new_depth = doc.sample_depth(word_idx)
            new_topic = doc.topics[new_depth]
            
            # increment count of words
            doc.depth_cnt_words[new_depth, word_idx] += 1
            if train: new_topic.cnt_words[word_idx] += 1
            doc.word_depths.append(new_depth) # for reference when sampling
            
        assert len(doc.word_depths) == len(doc.words) == np.sum(doc.depth_cnt_words)

In [137]:
def sample_word_topics(docs, train=True):
    for doc in docs:
        if doc.idx % 1000 == 0: print(doc.idx, end=' ')
        for word_index, word_idx in enumerate(doc.words):
            # refer depth of word
            old_depth = doc.word_depths[word_index]
            old_topic = doc.topics[old_depth]
            
            # decrement count of words
            doc.depth_cnt_words[old_depth, word_idx] -= 1
            if train: old_topic.cnt_words[word_idx] -= 1            
            
            # sample depth of word
            new_depth = doc.sample_depth(word_idx)
            new_topic = doc.topics[new_depth]
            
            # increment count of words
            doc.depth_cnt_words[new_depth, word_idx] += 1
            if train: new_topic.cnt_words[word_idx] += 1
            doc.word_depths[word_index] = new_depth # for sample
            
        assert len(doc.word_depths) == len(doc.words) == np.sum(doc.depth_cnt_words)

In [138]:
def get_prob_word_topics(docs, n_iter=1):
    init_word_topics(docs, train=False)
    for _ in range(n_iter):
        sample_word_topics(docs, train=False)
        
    for doc in docs:
        for word_index, word_idx in enumerate(doc.words):
            probs_depth = doc.get_probs_depth(word_idx)
            doc.probs_depths.append(probs_depth)

## check

In [139]:
def recur_cnt_words(topic):
    cnt_words = np.sum(topic.cnt_words)
    for child in topic.children:
        cnt_words += recur_cnt_words(child)
    return cnt_words
    
def assert_sum_cnt_words(topic_root):
    sum_cnt_words = recur_cnt_words(topic_root)
    assert sum_cnt_words == sum([len(doc.words) for doc in docs])

### init

In [140]:
alpha = np.array([10., 5., 1.])
gam = 0.01
eta = 1.
n_depth = 3
verbose = True

In [141]:
n_sample = 100
docs = [Doc(idx=doc_idx, words=doc_words, bow=doc_bow, n_depth=config.n_depth) for doc_idx, (doc_words, doc_bow) in enumerate(zip(docs_words, docs_bow))]
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, n_doc=n_doc, n_vocab=n_vocab)

In [142]:
init_doc_topics(docs=docs, topic_root=topic_root)

Depth:  0 p_child:  [1.] selected: 0
Depth:  1 p_child:  [1.] selected: 0
Depth:  0 p_child:  [0.99009901 0.00990099] selected: 0
Depth:  1 p_child:  [0.99009901 0.00990099] selected: 0
Depth:  0 p_child:  [0.99502488 0.00497512] selected: 0
Depth:  1 p_child:  [0.99502488 0.00497512] selected: 0
Depth:  0 p_child:  [0.99667774 0.00332226] selected: 0
Depth:  1 p_child:  [0.99667774 0.00332226] selected: 0
Depth:  0 p_child:  [0.99750623 0.00249377] selected: 0
Depth:  1 p_child:  [0.99750623 0.00249377] selected: 0
Depth:  0 p_child:  [0.99800399 0.00199601] selected: 0
Depth:  1 p_child:  [0.99800399 0.00199601] selected: 0
Depth:  0 p_child:  [0.99833611 0.00166389] selected: 0
Depth:  1 p_child:  [0.99833611 0.00166389] selected: 0
Depth:  0 p_child:  [0.99857347 0.00142653] selected: 0
Depth:  1 p_child:  [0.99857347 0.00142653] selected: 0
Depth:  0 p_child:  [0.99875156 0.00124844] selected: 0
Depth:  1 p_child:  [0.99875156 0.00124844] selected: 0
Depth:  0 p_child:  [0.9988901

Depth:  1 p_child:  [9.99925931e-01 7.40685875e-05] selected: 0
Depth:  0 p_child:  [9.99926476e-01 7.35240056e-05] selected: 0
Depth:  1 p_child:  [9.99926476e-01 7.35240056e-05] selected: 0
Depth:  0 p_child:  [9.99927013e-01 7.29873732e-05] selected: 0
Depth:  1 p_child:  [9.99927013e-01 7.29873732e-05] selected: 0
Depth:  0 p_child:  [9.99927541e-01 7.24585175e-05] selected: 0
Depth:  1 p_child:  [9.99927541e-01 7.24585175e-05] selected: 0
Depth:  0 p_child:  [9.99928063e-01 7.19372707e-05] selected: 0
Depth:  1 p_child:  [9.99928063e-01 7.19372707e-05] selected: 0
Depth:  0 p_child:  [9.99928577e-01 7.14234698e-05] selected: 0
Depth:  1 p_child:  [9.99928577e-01 7.14234698e-05] selected: 0
Depth:  0 p_child:  [9.99929083e-01 7.09169562e-05] selected: 0
Depth:  1 p_child:  [9.99929083e-01 7.09169562e-05] selected: 0
Depth:  0 p_child:  [9.99929582e-01 7.04175762e-05] selected: 0
Depth:  1 p_child:  [9.99929582e-01 7.04175762e-05] selected: 0
Depth:  0 p_child:  [9.99930075e-01 6.99

Depth:  0 p_child:  [9.99961687e-01 3.83127083e-05] selected: 0
Depth:  1 p_child:  [9.99961687e-01 3.83127083e-05] selected: 0
Depth:  0 p_child:  [9.99961834e-01 3.81664822e-05] selected: 0
Depth:  1 p_child:  [9.99961834e-01 3.81664822e-05] selected: 0
Depth:  0 p_child:  [9.99961979e-01 3.80213680e-05] selected: 0
Depth:  1 p_child:  [9.99961979e-01 3.80213680e-05] selected: 0
Depth:  0 p_child:  [9.99962123e-01 3.78773531e-05] selected: 0
Depth:  1 p_child:  [9.99962123e-01 3.78773531e-05] selected: 0
Depth:  0 p_child:  [9.99962266e-01 3.77344251e-05] selected: 0
Depth:  1 p_child:  [9.99962266e-01 3.77344251e-05] selected: 0
Depth:  0 p_child:  [9.99962407e-01 3.75925717e-05] selected: 0
Depth:  1 p_child:  [9.99962407e-01 3.75925717e-05] selected: 0
Depth:  0 p_child:  [9.99962548e-01 3.74517808e-05] selected: 0
Depth:  1 p_child:  [9.99962548e-01 3.74517808e-05] selected: 0
Depth:  0 p_child:  [9.99962688e-01 3.73120406e-05] selected: 0
Depth:  1 p_child:  [9.99962688e-01 3.73

Depth:  1 p_child:  [9.99975125e-01 2.48750031e-05] selected: 0
Depth:  0 p_child:  [9.99975187e-01 2.48132801e-05] selected: 0
Depth:  1 p_child:  [9.99975187e-01 2.48132801e-05] selected: 0
Depth:  0 p_child:  [9.99975248e-01 2.47518626e-05] selected: 0
Depth:  1 p_child:  [9.99975248e-01 2.47518626e-05] selected: 0
Depth:  0 p_child:  [9.99975309e-01 2.46907484e-05] selected: 0
Depth:  1 p_child:  [9.99975309e-01 2.46907484e-05] selected: 0
Depth:  0 p_child:  [9.99975370e-01 2.46299352e-05] selected: 0
Depth:  1 p_child:  [9.99975370e-01 2.46299352e-05] selected: 0
Depth:  0 p_child:  [9.99975431e-01 2.45694209e-05] selected: 0
Depth:  1 p_child:  [9.99975431e-01 2.45694209e-05] selected: 0
Depth:  0 p_child:  [9.99975491e-01 2.45092032e-05] selected: 0
Depth:  1 p_child:  [9.99975491e-01 2.45092032e-05] selected: 0
Depth:  0 p_child:  [9.99975551e-01 2.44492800e-05] selected: 0
Depth:  1 p_child:  [9.99975551e-01 2.44492800e-05] selected: 0
Depth:  0 p_child:  [9.9997561e-01 2.438

Depth:  0 p_child:  [9.99981550e-01 1.84498441e-05] selected: 0
Depth:  1 p_child:  [9.99981550e-01 1.84498441e-05] selected: 0
Depth:  0 p_child:  [9.99981584e-01 1.84158671e-05] selected: 0
Depth:  1 p_child:  [9.99981584e-01 1.84158671e-05] selected: 0
Depth:  0 p_child:  [9.99981618e-01 1.83820150e-05] selected: 0
Depth:  1 p_child:  [9.99981618e-01 1.83820150e-05] selected: 0
Depth:  0 p_child:  [9.99981652e-01 1.83482872e-05] selected: 0
Depth:  1 p_child:  [9.99981652e-01 1.83482872e-05] selected: 0
Depth:  0 p_child:  [9.99981685e-01 1.83146829e-05] selected: 0
Depth:  1 p_child:  [9.99981685e-01 1.83146829e-05] selected: 0
Depth:  0 p_child:  [9.99981719e-01 1.82812014e-05] selected: 0
Depth:  1 p_child:  [9.99981719e-01 1.82812014e-05] selected: 0
Depth:  0 p_child:  [9.99981752e-01 1.82478422e-05] selected: 0
Depth:  1 p_child:  [9.99981752e-01 1.82478422e-05] selected: 0
Depth:  0 p_child:  [9.99981785e-01 1.82146045e-05] selected: 0
Depth:  1 p_child:  [9.99981785e-01 1.82

...

Depth:  1 p_child:  [9.99997756e-01 2.24365662e-06] selected: 0
Depth:  0 p_child:  [9.99997757e-01 2.24315334e-06] selected: 0
Depth:  1 p_child:  [9.99997757e-01 2.24315334e-06] selected: 0
Depth:  0 p_child:  [9.99997757e-01 2.24265027e-06] selected: 0
Depth:  1 p_child:  [9.99997757e-01 2.24265027e-06] selected: 0
Depth:  0 p_child:  [9.99997758e-01 2.24214744e-06] selected: 0
Depth:  1 p_child:  [9.99997758e-01 2.24214744e-06] selected: 0
Depth:  0 p_child:  [9.99997758e-01 2.24164483e-06] selected: 0
Depth:  1 p_child:  [9.99997758e-01 2.24164483e-06] selected: 0
Depth:  0 p_child:  [9.99997759e-01 2.24114244e-06] selected: 0
Depth:  1 p_child:  [9.99997759e-01 2.24114244e-06] selected: 0
Depth:  0 p_child:  [9.99997759e-01 2.24064029e-06] selected: 0
Depth:  1 p_child:  [9.99997759e-01 2.24064029e-06] selected: 0
Depth:  0 p_child:  [9.99997760e-01 2.24013835e-06] selected: 0
Depth:  1 p_child:  [9.99997760e-01 2.24013835e-06] selected: 0
Depth:  0 p_child:  [9.99997760e-01 2.23

Depth:  1 p_child:  [9.99997818e-01 2.18244831e-06] selected: 0
Depth:  0 p_child:  [9.99997818e-01 2.18197211e-06] selected: 0
Depth:  1 p_child:  [9.99997818e-01 2.18197211e-06] selected: 0
Depth:  0 p_child:  [9.99997819e-01 2.18149611e-06] selected: 0
Depth:  1 p_child:  [9.99997819e-01 2.18149611e-06] selected: 0
Depth:  0 p_child:  [9.99997819e-01 2.18102032e-06] selected: 0
Depth:  1 p_child:  [9.99997819e-01 2.18102032e-06] selected: 0
Depth:  0 p_child:  [9.99997819e-01 2.18054474e-06] selected: 0
Depth:  1 p_child:  [9.99997819e-01 2.18054474e-06] selected: 0
Depth:  0 p_child:  [9.99997820e-01 2.18006937e-06] selected: 0
Depth:  1 p_child:  [9.99997820e-01 2.18006937e-06] selected: 0
Depth:  0 p_child:  [9.9999782e-01 2.1795942e-06] selected: 0
Depth:  1 p_child:  [9.9999782e-01 2.1795942e-06] selected: 0
Depth:  0 p_child:  [9.99997821e-01 2.17911924e-06] selected: 0
Depth:  1 p_child:  [9.99997821e-01 2.17911924e-06] selected: 0
Depth:  0 p_child:  [9.99997821e-01 2.178644

Depth:  1 p_child:  [9.99997876e-01 2.12449092e-06] selected: 0
Depth:  0 p_child:  [9.99997876e-01 2.12403967e-06] selected: 0
Depth:  1 p_child:  [9.99997876e-01 2.12403967e-06] selected: 0
Depth:  0 p_child:  [9.99997876e-01 2.12358861e-06] selected: 0
Depth:  1 p_child:  [9.99997876e-01 2.12358861e-06] selected: 0
Depth:  0 p_child:  [9.99997877e-01 2.12313774e-06] selected: 0
Depth:  1 p_child:  [9.99997877e-01 2.12313774e-06] selected: 0
Depth:  0 p_child:  [9.99997877e-01 2.12268707e-06] selected: 0
Depth:  1 p_child:  [9.99997877e-01 2.12268707e-06] selected: 0
Depth:  0 p_child:  [9.99997878e-01 2.12223658e-06] selected: 0
Depth:  1 p_child:  [9.99997878e-01 2.12223658e-06] selected: 0
Depth:  0 p_child:  [9.99997878e-01 2.12178629e-06] selected: 0
Depth:  1 p_child:  [9.99997878e-01 2.12178629e-06] selected: 0
Depth:  0 p_child:  [9.99997879e-01 2.12133619e-06] selected: 0
Depth:  1 p_child:  [9.99997879e-01 2.12133619e-06] selected: 0
Depth:  0 p_child:  [9.99997879e-01 2.12

Depth:  1 p_child:  [9.99997930e-01 2.06953214e-06] selected: 0
Depth:  0 p_child:  [9.99997931e-01 2.06910393e-06] selected: 0
Depth:  1 p_child:  [9.99997931e-01 2.06910393e-06] selected: 0
Depth:  0 p_child:  [9.99997931e-01 2.06867590e-06] selected: 0
Depth:  1 p_child:  [9.99997931e-01 2.06867590e-06] selected: 0
Depth:  0 p_child:  [9.99997932e-01 2.06824805e-06] selected: 0
Depth:  1 p_child:  [9.99997932e-01 2.06824805e-06] selected: 0
Depth:  0 p_child:  [9.99997932e-01 2.06782037e-06] selected: 0
Depth:  1 p_child:  [9.99997932e-01 2.06782037e-06] selected: 0
Depth:  0 p_child:  [9.99997933e-01 2.06739287e-06] selected: 0
Depth:  1 p_child:  [9.99997933e-01 2.06739287e-06] selected: 0
Depth:  0 p_child:  [9.99997933e-01 2.06696555e-06] selected: 0
Depth:  1 p_child:  [9.99997933e-01 2.06696555e-06] selected: 0
Depth:  0 p_child:  [9.99997933e-01 2.06653840e-06] selected: 0
Depth:  1 p_child:  [9.99997933e-01 2.06653840e-06] selected: 0
Depth:  0 p_child:  [9.99997934e-01 2.06

Depth:  1 p_child:  [9.99997983e-01 2.01734513e-06] selected: 0
Depth:  0 p_child:  [9.99997983e-01 2.01693825e-06] selected: 0
Depth:  1 p_child:  [9.99997983e-01 2.01693825e-06] selected: 0
Depth:  0 p_child:  [9.99997983e-01 2.01653153e-06] selected: 0
Depth:  1 p_child:  [9.99997983e-01 2.01653153e-06] selected: 0
Depth:  0 p_child:  [9.99997984e-01 2.01612497e-06] selected: 0
Depth:  1 p_child:  [9.99997984e-01 2.01612497e-06] selected: 0
Depth:  0 p_child:  [9.99997984e-01 2.01571857e-06] selected: 0
Depth:  1 p_child:  [9.99997984e-01 2.01571857e-06] selected: 0
Depth:  0 p_child:  [9.99997985e-01 2.01531234e-06] selected: 0
Depth:  1 p_child:  [9.99997985e-01 2.01531234e-06] selected: 0
Depth:  0 p_child:  [9.99997985e-01 2.01490628e-06] selected: 0
Depth:  1 p_child:  [9.99997985e-01 2.01490628e-06] selected: 0
Depth:  0 p_child:  [9.99997985e-01 2.01450037e-06] selected: 0
Depth:  1 p_child:  [9.99997985e-01 2.01450037e-06] selected: 0
Depth:  0 p_child:  [9.99997986e-01 2.01

In [143]:
init_word_topics(docs=docs)
assert_sum_cnt_words(topic_root)

0 1000 2000 3000 4000 

### sample

In [144]:
sample_doc_topics(docs=docs, topic_root=topic_root)
assert_sum_cnt_words(topic_root)

Depth:  0 p_child:  [0.5001132 0.4998868] selected: 1
Depth:  1 p_child:  [1.] selected: 0
Depth:  0 p_child:  [9.96621397e-01 3.34327150e-03 3.53315570e-05] selected: 0
Depth:  1 p_child:  [0.98715997 0.01284003] selected: 0
Depth:  0 p_child:  [0.2138142  0.77790306 0.00828275] selected: 0
Depth:  1 p_child:  [0.05671263 0.94328737] selected: 1
Depth:  0 p_child:  [9.99946861e-01 5.25907496e-05 5.48491511e-07] selected: 0
Depth:  1 p_child:  [9.86666499e-01 1.32014857e-02 1.32014857e-04] selected: 0
Depth:  0 p_child:  [9.99882365e-01 1.16397665e-04 1.23702887e-06] selected: 0
Depth:  1 p_child:  [9.91685413e-01 8.23226478e-03 8.23226478e-05] selected: 0
Depth:  0 p_child:  [1.41296627e-04 9.89010451e-01 1.08482528e-02] selected: 1
Depth:  1 p_child:  [0.99009901 0.00990099] selected: 0
Depth:  0 p_child:  [0.01584824 0.98083339 0.00331837] selected: 1
Depth:  1 p_child:  [0.99502488 0.00497512] selected: 0
Depth:  0 p_child:  [0.01239767 0.98458716 0.00301517] selected: 1
Depth:  1 

Depth:  0 p_child:  [9.99948091e-01 5.19066724e-05 2.05226576e-09] selected: 0
Depth:  1 p_child:  [9.91823458e-01 7.66637535e-03 5.09290456e-04 8.76143159e-07] selected: 0
Depth:  0 p_child:  [3.13298018e-01 6.86656947e-01 4.50354969e-05] selected: 1
Depth:  1 p_child:  [9.99694436e-01 3.05563616e-04] selected: 0
Depth:  0 p_child:  [9.70922659e-01 2.90694253e-02 7.91541126e-06] selected: 0
Depth:  1 p_child:  [9.29092583e-01 6.25786936e-02 8.31441977e-03 1.43034724e-05] selected: 0
Depth:  0 p_child:  [7.64830287e-01 2.35151255e-01 1.84582041e-05] selected: 0
Depth:  1 p_child:  [8.14619386e-01 1.60382864e-01 2.49539546e-02 4.37956232e-05] selected: 0
Depth:  0 p_child:  [9.54129835e-01 4.58647648e-02 5.40023393e-06] selected: 0
Depth:  1 p_child:  [1.77960866e-03 7.60997670e-01 2.36807111e-01 4.15610078e-04] selected: 1
Depth:  0 p_child:  [2.66354037e-05 9.99787577e-01 1.85787291e-04] selected: 1
Depth:  1 p_child:  [9.99360324e-01 6.39675588e-04] selected: 0
Depth:  0 p_child:  [9

Depth:  0 p_child:  [6.00400269e-01 3.99567593e-01 3.21382508e-05] selected: 0
Depth:  1 p_child:  [3.98423278e-01 4.64615443e-01 1.36910224e-01 5.10554990e-05] selected: 0
Depth:  0 p_child:  [9.97951861e-01 2.04761833e-03 5.20795451e-07] selected: 0
Depth:  1 p_child:  [6.93517583e-07 8.01810567e-01 1.98030145e-01 1.58593907e-04] selected: 2
Depth:  0 p_child:  [9.98026423e-01 1.97357645e-03 5.62400544e-10] selected: 0
Depth:  1 p_child:  [9.79624084e-01 1.41691993e-02 6.20441059e-03 2.30604500e-06] selected: 0
Depth:  0 p_child:  [3.49667392e-01 6.50327419e-01 5.18897853e-06] selected: 0
Depth:  1 p_child:  [7.42177692e-02 6.43779303e-01 2.81898153e-01 1.04775436e-04] selected: 1
Depth:  0 p_child:  [9.69841455e-01 3.01585404e-02 4.18905847e-09] selected: 0
Depth:  1 p_child:  [5.50073762e-01 3.89946360e-01 5.99553540e-02 2.45239606e-05] selected: 1
Depth:  0 p_child:  [8.62395079e-01 1.37590165e-01 1.47556419e-05] selected: 1
Depth:  1 p_child:  [9.99714658e-01 2.85342100e-04] sele

Depth:  0 p_child:  [9.98091789e-01 1.90783995e-03 3.71125754e-07] selected: 0
Depth:  1 p_child:  [9.52586050e-01 3.92408084e-02 8.16949451e-03 3.64723993e-06] selected: 0
Depth:  0 p_child:  [7.90353421e-03 9.91178462e-01 9.18003688e-04] selected: 1
Depth:  1 p_child:  [9.99827101e-01 1.72898640e-04] selected: 0
Depth:  0 p_child:  [9.74821495e-01 2.51784201e-02 8.52459141e-08] selected: 0
Depth:  1 p_child:  [3.53688946e-02 8.53676799e-01 1.10903065e-01 5.12409412e-05] selected: 2
Depth:  0 p_child:  [2.46217984e-08 9.99998584e-01 1.39098648e-06] selected: 1
Depth:  1 p_child:  [9.99802027e-01 1.97973392e-04] selected: 0
Depth:  0 p_child:  [9.99995383e-01 4.61607050e-06 1.12114809e-09] selected: 0
Depth:  1 p_child:  [1.07937411e-02 8.89939514e-01 9.92536756e-02 1.30691642e-05] selected: 1
Depth:  0 p_child:  [9.99243483e-01 7.55543509e-04 9.73037022e-07] selected: 0
Depth:  1 p_child:  [9.99525922e-01 2.85991759e-04 1.88042850e-04 4.38151959e-08] selected: 0
Depth:  0 p_child:  [9

Depth:  1 p_child:  [7.61342882e-03 7.51371172e-01 2.40988996e-01 2.64034222e-05] selected: 1
Depth:  0 p_child:  [9.92767182e-01 7.23029853e-03 2.51987759e-06] selected: 0
Depth:  1 p_child:  [1.36129583e-01 5.24576864e-01 3.39162333e-01 1.31218941e-04] selected: 0
Depth:  0 p_child:  [9.81156074e-01 1.88439075e-02 1.80648137e-08] selected: 0
Depth:  1 p_child:  [8.00563327e-02 7.11545067e-01 2.08354464e-01 4.41354594e-05] selected: 1
Depth:  0 p_child:  [9.99830627e-01 1.69371442e-04 1.48833296e-09] selected: 0
Depth:  1 p_child:  [8.44368654e-03 9.47684939e-01 4.38617635e-02 9.61123269e-06] selected: 2
Depth:  0 p_child:  [7.22466632e-01 2.77526939e-01 6.42872184e-06] selected: 0
Depth:  1 p_child:  [2.90567311e-01 6.17678463e-01 9.17186932e-02 3.55325216e-05] selected: 1
Depth:  0 p_child:  [5.20150112e-01 4.79755946e-01 9.39412857e-05] selected: 1
Depth:  1 p_child:  [9.99910185e-01 8.98154036e-05] selected: 0
Depth:  0 p_child:  [9.97482150e-01 2.51768456e-03 1.65122695e-07] sele

...

Depth:  0 p_child:  [8.47372101e-01 3.64546508e-06 2.89601521e-04 3.30241202e-03
 1.49032240e-01 6.27129502e-10] selected: 0
Depth:  1 p_child:  [3.36875262e-01 5.10892909e-01 1.52231255e-01 5.73870567e-07] selected: 0
Depth:  0 p_child:  [1.49101594e-01 2.39690465e-03 3.90035268e-02 8.06015609e-01
 3.48085521e-03 1.51046226e-06] selected: 3
Depth:  1 p_child:  [6.19078057e-01 3.80717782e-01 2.04160698e-04] selected: 0
Depth:  0 p_child:  [9.05359710e-01 2.70996269e-03 6.03811251e-03 8.50920886e-02
 8.00110117e-04 1.56629062e-08] selected: 0
Depth:  1 p_child:  [6.46941852e-01 2.80400603e-01 7.26569822e-02 5.62716450e-07] selected: 0
Depth:  0 p_child:  [2.34141034e-06 1.66664072e-11 5.22875973e-05 9.75306949e-01
 2.45766747e-02 6.17474517e-05] selected: 3
Depth:  1 p_child:  [1.77604730e-02 9.81286769e-01 9.52758188e-04] selected: 1
Depth:  0 p_child:  [1.20776929e-01 9.81933080e-03 3.24041217e-01 2.50884332e-01
 2.94452758e-01 2.54342227e-05] selected: 3
Depth:  1 p_child:  [9.904698

Depth:  1 p_child:  [9.80768016e-01 1.91641396e-02 6.78443652e-05] selected: 0
Depth:  0 p_child:  [8.69920600e-02 2.25133280e-05 1.05771710e-02 9.00069711e-01
 2.33853992e-03 4.52599814e-09] selected: 3
Depth:  1 p_child:  [9.40809765e-01 5.91875233e-02 2.71139364e-06] selected: 0
Depth:  0 p_child:  [8.57468639e-03 3.91238780e-07 4.40649545e-06 1.35466315e-03
 9.90061909e-01 3.94350281e-06] selected: 4
Depth:  1 p_child:  [9.99685012e-01 3.14987956e-04] selected: 0
Depth:  0 p_child:  [1.07401581e-02 1.38701387e-10 3.44013152e-06 1.68189168e-02
 9.72437241e-01 2.44024922e-07] selected: 4
Depth:  1 p_child:  [0.99826522 0.00173478] selected: 0
Depth:  0 p_child:  [9.59503335e-01 2.29561503e-02 8.94009345e-04 1.63467376e-02
 2.99767361e-04 4.53999017e-10] selected: 3
Depth:  1 p_child:  [8.75189238e-01 1.24769323e-01 4.14387327e-05] selected: 0
Depth:  0 p_child:  [6.80388611e-01 4.33890466e-04 1.64575604e-03 1.89179979e-01
 1.28351742e-01 2.17611511e-08] selected: 0
Depth:  1 p_child:

Depth:  1 p_child:  [3.97290430e-03 6.92397745e-01 3.03441473e-01 1.87877702e-04] selected: 1
Depth:  0 p_child:  [5.37326997e-01 3.50329348e-02 3.81276676e-02 1.36240949e-01
 2.53269057e-01 2.39534360e-06] selected: 0
Depth:  1 p_child:  [4.89174759e-01 3.64048304e-02 4.74412569e-01 7.84192233e-06] selected: 2
Depth:  0 p_child:  [9.98541968e-01 2.53915345e-08 4.23715057e-06 7.21230538e-04
 7.32539198e-04 7.97128285e-16] selected: 0
Depth:  1 p_child:  [2.58272471e-01 7.39593928e-01 2.13315793e-03 4.42787868e-07] selected: 1
Depth:  0 p_child:  [9.88257738e-01 8.68432593e-03 1.25943742e-03 1.68884633e-03
 1.09651914e-04 4.96302761e-11] selected: 0
Depth:  1 p_child:  [7.00195965e-02 8.71131957e-01 5.88471155e-02 1.33076412e-06] selected: 1
Depth:  0 p_child:  [1.09404925e-01 4.10783070e-04 4.91684854e-04 7.89052762e-01
 1.00639745e-01 9.94629872e-08] selected: 3
Depth:  1 p_child:  [5.42906271e-01 4.57064581e-01 2.91489164e-05] selected: 0
Depth:  0 p_child:  [4.30914037e-07 5.6255773

Depth:  0 p_child:  [9.98752948e-01 1.48210013e-06 1.50884449e-04 9.04389236e-04
 1.90295996e-04 2.80952587e-10] selected: 0
Depth:  1 p_child:  [2.54942663e-01 7.38543758e-01 6.50328520e-03 1.02931173e-05] selected: 1
Depth:  0 p_child:  [1.64636073e-01 3.83543809e-01 1.50943141e-01 5.33879056e-02
 2.47488878e-01 1.93008984e-07] selected: 4
Depth:  1 p_child:  [9.99950345e-01 4.96547827e-05] selected: 0
Depth:  0 p_child:  [1.19853366e-01 9.95155495e-05 9.01575702e-04 4.86017042e-01
 3.93128124e-01 3.76135241e-07] selected: 3
Depth:  1 p_child:  [9.96357585e-01 3.63912996e-03 3.28489238e-06] selected: 0
Depth:  0 p_child:  [9.60318782e-01 2.05806783e-03 3.32334221e-02 4.35902660e-03
 3.06981612e-05 3.52201426e-09] selected: 0
Depth:  1 p_child:  [5.16535334e-02 3.42817947e-01 6.05523570e-01 4.94956770e-06] selected: 2
Depth:  0 p_child:  [9.48805918e-01 3.53738693e-04 8.46093575e-03 2.26322171e-02
 1.97468439e-02 3.46759401e-07] selected: 3
Depth:  1 p_child:  [9.03071762e-01 9.691113

Depth:  1 p_child:  [0.99868574 0.00131426] selected: 0
Depth:  0 p_child:  [1.91136544e-02 2.18680252e-02 2.57458235e-01 6.98585518e-01
 2.97439554e-03 1.72199644e-07] selected: 3
Depth:  1 p_child:  [6.24007064e-01 3.75977209e-01 1.57273145e-05] selected: 0
Depth:  0 p_child:  [7.20484112e-02 9.26637024e-01 1.29002614e-03 2.13692499e-05
 3.16884029e-06 8.58032780e-10] selected: 1
Depth:  1 p_child:  [1.37215131e-02 9.86213383e-01 6.51036326e-05] selected: 1
Depth:  0 p_child:  [2.40422309e-04 7.24825252e-10 2.53420370e-05 9.99732520e-01
 1.65847672e-06 5.66903329e-08] selected: 3
Depth:  1 p_child:  [5.62199536e-01 4.37703341e-01 9.71229346e-05] selected: 0
Depth:  0 p_child:  [1.39497547e-01 2.44537379e-02 8.32690211e-01 3.19629413e-03
 1.56688894e-04 5.52074402e-06] selected: 2
Depth:  1 p_child:  [0.46538356 0.53388785 0.0007286 ] selected: 1
Depth:  0 p_child:  [8.51404095e-02 3.93146198e-01 5.19710324e-01 2.05534234e-04
 1.78714155e-03 1.03929389e-05] selected: 0
Depth:  1 p_chi

In [145]:
sample_word_topics(docs=docs)
assert_sum_cnt_words(topic_root)

0 1000 2000 3000 4000 

### test

In [146]:
test_docs = [Doc(idx=doc_idx, words=doc_words, bow=doc_bow, n_depth=config.n_depth) for doc_idx, (doc_words, doc_bow) in enumerate(zip(test_docs_words, test_docs_bow))]

In [147]:
get_prob_doc_topics(test_docs, topic_root)
assert (np.sum([sum(probs_path.values()) for probs_path in test_docs[0].probs_paths]) > n_depth-1e-5) and (np.sum([sum(probs_path.values()) for probs_path in test_docs[0].probs_paths]) < n_depth+1e-5)

In [149]:
get_prob_word_topics(test_docs, n_iter=1)

0 

AssertionError: 

In [150]:
%debug

> <ipython-input-131-4ba725f0f857>(20)get_probs_depth()
     18         s_words = np.array([topic.cnt_words[word_idx] for topic in self.topics]) + eta # Depth
     19         z_words = np.array([np.sum(topic.cnt_words) for topic in self.topics]) + n_vocab*eta # Depth
---> 20         assert s_docs.shape == s_words.shape == z_words.shape
     21 
     22         s_depths = s_docs*s_words/z_words

ipdb> s_docs.shape
(3,)
ipdb> s_words.shape
(0,)
ipdb> q


In [148]:
assert_sum_cnt_words(topic_root)

### print tree

In [112]:
def print_child_idxs(topic):
    topic_freq_words = [idx_to_word[bow_idxs[bow_index]] for bow_index in np.argsort(topic.cnt_words)[::-1][:10]]
    print('  '*topic.depth, topic.idx, ':', [child.idx for child in topic.children], topic.cnt_doc, np.sum(topic.cnt_words), topic_freq_words)
    for topic in topic.children:
        print_child_idxs(topic)
print_child_idxs(topic_root)

 0 : ['0-1', '0-2', '0-3', '0-4', '0-5'] 5000 56834.0 ['!', 'carry', 'sleeve', 'bought', 'pocket', 'room', 'pockets', 'quality', '-', 'zipper']
   0-1 : ['0-1-1', '0-1-2', '0-1-3'] 3655 20227.0 ['inside', 'pro', 'nice', 'perfect', '!', 'size', '...', 'durable', 'hard', 'netbook']
     0-1-1 : [] 1527 1698.0 ['smell', 'mbp', 'aluminum', 'front', 'inside', 'pocket', 'sleeves', 'protecting', 'flap', 'thin']
     0-1-2 : [] 1852 1830.0 ['stores', 'heavy', 'tons', 'year', 'love', 'features', 'pack', 'tear', 'side', '!']
     0-1-3 : [] 276 418.0 ['colors', 'black', 'scratches', 'keyboard', 'excellent', 'website', 'bit', 'disappointing', 'remove', 'snapped']
   0-2 : ['0-2-1', '0-2-2'] 857 5604.0 ['years', 'year', 'handle', 'top', 'bags', "'m", 'broke', 'months', 'zippers', 'return']
     0-2-1 : [] 726 1023.0 ['plastic', 'owned', 'zipper', 'weeks', 'foam', 'description', 'pictures', 'thin', 'clip', 'totally']
     0-2-2 : [] 131 221.0 ['found', 'start', 'months', 'walk', 'dirty', 'durable',

## run

In [ ]:
n_sample = 100
docs = [Doc(idx=doc_idx, words=doc_words, bow=doc_bow, n_depth=config.n_depth) for doc_idx, (doc_words, doc_bow) in enumerate(zip(docs_words, docs_bow))]
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, n_doc=n_doc, n_vocab=n_vocab)
for i_sample in range(n_sample):
    print(i_sample)
    # sample path of doc
    doc_topics = sample_doc_topics(docs=docs, doc_topics=doc_topics, doc_depth_words=doc_depth_words, topic_root=topic_root)
    
    # assign words to each topic
    doc_word_indices_depth, doc_depth_words = sample_word_topics(docs=docs, doc_topics=doc_topics, doc_word_indices_depth=doc_word_indices_depth)

## print 